In [5]:
import pandas as pd

In [6]:
score = pd.read_csv("score.tsv", delimiter="\t")

In [29]:
import subprocess
from subprocess import Popen, PIPE, STDOUT
from pathlib import Path

def make_script(video_path, keypoint_path, force=False):
    
    if not force and keypoint_path.is_file():
        print("Keypoint Exist - ", keypoint_path)
        return

    cmd = [
        "python", "demo/top_down_video_demo_with_mmdet.py",
        "demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py",
        "https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth",    
        "configs/body/2d_kpt_sview_rgb_vid/posewarper/posetrack18/hrnet_w48_posetrack18_384x288_posewarper_stage2.py",
        "https://download.openmmlab.com/mmpose/top_down/posewarper/hrnet_w48_posetrack18_384x288_posewarper_stage2-4abf88db_20211130.pth",
        "--video-path", video_path,
        "--keypoint-path", keypoint_path,
        "--out-video-root", "vis2d_bt_results",
        "--use-multi-frames",
        "--online",
    ]
    
    return cmd

In [30]:
processes = []

In [41]:

import os, re, mmcv

test_cnt = 10
cmd_d = {}
basedir = '/mnt/c/Users/USER/Dropbox/머신러닝_수강생영상 (2)/'
for i, r in score.iterrows():
    title = r['title'].strip()
    filename = r['file name']
    score1 = r['Skill']
    score2 = r['Expression']

    title = re.sub(r"[\*\?]", "", title)
    filename = re.sub(r"[\*\?]", "", filename)

    found = False
    dir1 = Path(basedir) / title
    for file in os.listdir(dir1):
        if file.startswith(filename) and not file.endswith("json"):
            filename = file
            found = True

    if not found:
        print(i, "Video Not Found", dir1 / filename)
        continue


    x_kp_path = (Path("vis2d_bt_results") / filename).with_suffix(".json")
    x_path = dir1 / filename
    
    found = False
    dir2 = dir1 / "강사님 영상"
    for file in os.listdir(dir2):
        if not file.startswith(".") and not file.endswith("json"):
            y_path = dir2 / file
            y_kp_path = (Path("vis2d_bt_results") / file).with_suffix(".json")
            found = True
    
    if not found:
        print(i, "Teacher Video Not Found", dir2)
        continue
    vid_x = mmcv.VideoReader(str(x_path))
    vid_y = mmcv.VideoReader(str(y_path))
    if abs(vid_x.fps - vid_y.fps) > 1:
        print(i, "FPS Problem", vid_x.fps, vid_y.fps)
        continue

    cmd = make_script(x_path, x_kp_path)
    if cmd is not None:
        cmd_d[x_path] = cmd
        
        
    cmd = make_script(y_path, y_kp_path)
    if cmd is not None:
        cmd_d[y_path] = cmd
    test_cnt -= 1
    if test_cnt == 0:
        break

Keypoint Exist -  vis2d_bt_results/0001_TWICE_Alcohol-Free (Pt.2).json
Keypoint Exist -  vis2d_bt_results/2022-06-15 - Instructor Twice Alcohol Free (Pt.2)1.json
1 FPS Problem 24.0 30.0
Keypoint Exist -  vis2d_bt_results/0003_NCT U_Baby Don't Stop (Original).json
Keypoint Exist -  vis2d_bt_results/Instructor_NCT U_Baby Don't Stop (Original)1.json
Keypoint Exist -  vis2d_bt_results/0004_NCT U_Baby Don't Stop (Original).json
Keypoint Exist -  vis2d_bt_results/Instructor_NCT U_Baby Don't Stop (Original)1.json
Keypoint Exist -  vis2d_bt_results/Instructor_NCT U_Baby Don't Stop (Original)1.json
Keypoint Exist -  vis2d_bt_results/0007_BAEKHYUN_Bambi.json
Keypoint Exist -  vis2d_bt_results/Instructor_BAEKHYUN_Bambi1.json
Keypoint Exist -  vis2d_bt_results/0008_BAEKHYUN_Bambi.json
Keypoint Exist -  vis2d_bt_results/Instructor_BAEKHYUN_Bambi1.json
Keypoint Exist -  vis2d_bt_results/0009_BAEKHYUN_Bambi.json
Keypoint Exist -  vis2d_bt_results/Instructor_BAEKHYUN_Bambi1.json
Keypoint Exist -  vis2

In [39]:
keys = list(cmd_d.keys())
len(keys)

14

In [40]:
import time

current_running = []
max_run = 3
while(len(keys) > 0):
    to_remove = []
    for p in current_running:
        if p.poll() is not None:
            to_remove.append(p)
    print(len(to_remove))
    if len(to_remove):
        for p in to_remove:
            current_running.remove(p)
    else:
        time.sleep(5)
    while(len(current_running) < max_run and len(keys) > 0):
        k = keys.pop()
        p = Popen(cmd_d[k])
        current_running.append(p)
for p in current_running:
    p.wait()

0
0
Initializing model...
Initializing model...
Initializing model...
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
0
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/posewarper/hrnet_w48_posetrack18_384x288_posewarper_stage2-4abf88db_20211130.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/posewarper/hrnet_w48_posetrack18_384x288_posewarper_stage2-4abf88db_20211130.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/posewarper/hrnet_w4

Traceback (most recent call last):
  File "/home/syd/workspace/mmpose/demo/top_down_video_demo_with_mmdet.py", line 209, in <module>
    main()
  File "/home/syd/workspace/mmpose/demo/top_down_video_demo_with_mmdet.py", line 101, in main
    det_model = init_detector(
  File "/home/syd/anaconda3/lib/python3.9/site-packages/mmdet/apis/inference.py", line 55, in init_detector
    model.to(device)
  File "/home/syd/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 987, in to
    return self._apply(convert)
  File "/home/syd/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 639, in _apply
    module._apply(fn)
  File "/home/syd/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 639, in _apply
    module._apply(fn)
  File "/home/syd/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 662, in _apply
    param_applied = fn(param)
  File "/home/syd/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.

[>>>>                        ] 185/1166, 1.0 task/s, elapsed: 179s, ETA:   947s1
0
[>>>>>>>                     ] 301/1176, 1.0 task/s, elapsed: 287s, ETA:   836sInitializing model...
0
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
[>>>>>>>                     ] 307/1176, 1.1 task/s, elapsed: 292s, ETA:   825sload checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/posewarper/hrnet_w48_posetrack18_384x288_posewarper_stage2-4abf88db_20211130.pth
[>>>>>>>                     ] 308/1176, 1.1 task/s, elapsed: 292s, ETA:   824s0
[>>>>                        ] 202/1166, 1.1 task/s, elapsed: 190s, ETA:   907sRunning inference...
[>>>>>>>                     ] 315/1176, 1.1 task/s, elapsed: 298s, ETA:   813s0
[>>>>>>>                     ] 320/1176, 1.1 task/s, elapsed: 303s, ETA:   809s0
[>>>>>>>                     ] 325/1176, 1.1 task/s, elapsed: 

In [37]:
for p in current_running:
    p.kill()

[>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 987/1058, 1.2 task/s, elapsed: 852s, ETA:    61s

[>>>                          ] 131/956, 1.1 task/s, elapsed: 122s, ETA:   766s

In [135]:
for p in current_running:
    p.wait()

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1176/1176, 5.4 task/s, elapsed: 217s, ETA:     0s
Stage 2: 2D-to-3D pose lifting.
Initializing model...
load checkpoint from http path: https://download.openmmlab.com/mmpose/body3d/videopose/videopose_h36m_243frames_fullconv_supervised_cpn_ft-88f5abbb_20210527.pth
Running 2D-to-3D pose lifting inference...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1176/1176, 12.2 task/s, elapsed: 96s, ETA:     0s


In [ ]:
화면에 발 끝이 잘리면 안됨
춤을 끝까지 다 춰야함

강사님 영상에서 시작점 끝점 하드코딩해서 체크
가만히 있는 구간도 점수에 포함되니까 움직이는 부분만 체크해서
